# W-CDNet Demo Notebook
This notebook allows you to train and test W-CDNet on the AICD dataset. <br>
The models are not trained on the full-res images from the AICD dataset, but on randomly cropped images. The images are automatically downloaded before training or testing. <br>

Before executing this notebook, change your runtime in google colab such that it uses GPU support (Runtime > Change runtime type > Hardware accelerator > GPU).

## Clone the W-CDNet repository, install requirements, set up environment
(Installing the requirements might throw some errors, but you can still continue...)

In [ ]:
import os
# Clone dev branch. 
!git clone -b main https://github.com/PhiAbs/W-CDNet.git

# Install requirements
os.chdir('W-CDNet')
!pip install -q -r requirements_gpu.txt

# Set environment variables. Required for segmentation-models repo.
%env SM_FRAMEWORK=tf.keras

# Install CRF-RNN layer

In [ ]:
os.chdir('Demo')
!git clone https://github.com/PhiAbs/crfasrnn_keras.git
os.chdir('crfasrnn_keras/src/cpp')
!make
os.chdir('../../..')

# Download train, val and test images

In [ ]:
!gdown https://drive.google.com/uc?id=1HLa4xpUZBcK_1__24_QuW3YPFRI_H5rF
!unzip -q AICD_strong_shadows_incl_no_change.zip

# Train the model
First you have to pretrain a model (use *params_pretraining.yaml* parameters). If you've done that you can finetune that model by using the *params_finetuning.yaml* parameters. Please check the parameter files and adjust them if necessary. <br> 
Please be aware that finetuning is quite slow since the crf-rnn layer cannot be trained with GPU. Pretraining however (without crf-rnn layer) does use GPU.

In [ ]:
# Set parameters
run_training = False

# params_pretraining.yaml if you want to perform pretraining
# or params_finetuning.yaml if you want to finetune a pretrained model
params_file = 'params_pretraining.yaml'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/W-CDNet/Demo/log/tensorboard

In [ ]:
if run_training:
  import train

  TrainWNet = train.TrainWNet(params_file=params_file)

  TrainWNet.execute_training()


# Test the model
You can either use weights from a model that you just trained with the above cell or you can download pretrained weights and use them instead. You'll have to adjust the parameters in the file **params_test.yaml** accordingly. <br>

Set `run_test` to True in the cell below if you want to see the change masks that a trained model predicts for images from the test set. The change mask is only shown if either the predicted or the ground truth image-level label belongs to one of the *changed* classes (*NOT* class 8). <br>

Please be aware that testing a finetuned model is quite slow since the crf-rnn layer does not use GPU support. The pre-trained models, however, have full GPU support.

In [ ]:
run_test = True
demo_weights_path = 'models/demo_weights'
params_file = 'params_test.yaml'

### Download pretrained weights

In [ ]:
if not os.path.isdir(demo_weights_path):
  os.makedirs(demo_weights_path)

os.chdir(demo_weights_path)
!gdown https://drive.google.com/uc?id=1NgZ1C57qZjOr5K7oFd3Uvye8dl6TWCiK
!gdown https://drive.google.com/uc?id=1VrQ6lttgKlBxZpnIYqQ2zZw-tPevDACE
os.chdir('../..')

### Start test loop


In [ ]:
if run_test:
  import test

  TestWNet = test.TestWNet(params_file=params_file)

  TestWNet.execute_test()